# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

from IPython.display import Image
from IPython.core.display import HTML 

from razdel import tokenize

In [2]:
data = pd.read_csv('labeled.csv')

In [3]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)

In [4]:
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [5]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 

In [6]:
y = train.toxic.values
y_test = test.toxic.values

In [7]:
razdel_tokenizer = lambda doc: [t.text for t in list(tokenize(doc))]
razdel_vectorizer = CountVectorizer(tokenizer=razdel_tokenizer)
X2 = razdel_vectorizer.fit_transform(train.comment)
X2_test = razdel_vectorizer.transform(test.comment) 

In [13]:
dt = DecisionTreeClassifier(max_depth=8, class_weight='balanced')
dt.fit(X, y)
pred = dt.predict(X_test)

In [14]:
dt2 = DecisionTreeClassifier(max_depth=8, class_weight='balanced')
dt2.fit(X2, y)
pred2 = dt2.predict(X2_test)

In [15]:
print('default classifier:')
print(classification_report(y_test, pred))
print('\n\nrazdel classifier:')
print(classification_report(y_test, pred2))

default classifier:
              precision    recall  f1-score   support

         0.0       0.80      0.58      0.67       951
         1.0       0.47      0.72      0.57       491

    accuracy                           0.63      1442
   macro avg       0.63      0.65      0.62      1442
weighted avg       0.69      0.63      0.64      1442



razdel classifier:
              precision    recall  f1-score   support

         0.0       0.77      0.72      0.74       951
         1.0       0.52      0.58      0.55       491

    accuracy                           0.67      1442
   macro avg       0.64      0.65      0.65      1442
weighted avg       0.68      0.67      0.68      1442



Раздел работает лучше, чем дефолтный токенизатор, но разница незначительная.

## Задание 2 (3 балла)

Преобразуйте таблицу с абсолютными частотностями в семинарской тетрадке в таблицу с tfidf значениями. (Таблица - https://i.ibb.co/r5Nc2HC/abs-bow.jpg) Формула tfidf есть в семинаре на картнике с пояснениями на английском. 
Считать нужно в питоне. Формат итоговой таблицы может быть любым, главное, чтобы был код и можно было воспроизвести вычисления. 

In [16]:
freqs = [[1, 1, 1, 0, 0, 0], 
         [1, 1, 1, 0, 0, 0], 
         [3, 0, 1, 1, 0, 0], 
         [1, 0, 0, 1, 1, 0], 
         [0, 0, 0, 0, 0, 1]]

In [17]:
table = pd.DataFrame(freqs, columns = ['я', 'ты', 'и', 'только', 'не', 'он'], index = ['я и ты','ты и я','я, я и только я', 'только не я', 'он'])
table

,я,ты,и,только,не,он
я и ты,1,1,1,0,0,0
ты и я,1,1,1,0,0,0
"я, я и только я",3,0,1,1,0,0
только не я,1,0,0,1,1,0
он,0,0,0,0,0,1


In [18]:
tfidf = table.apply(lambda x: x/ x.sum(), axis = 1)
tfidf

,я,ты,и,только,не,он
я и ты,0.333333,0.333333,0.333333,0.000000,0.000000,0.0
ты и я,0.333333,0.333333,0.333333,0.000000,0.000000,0.0
"я, я и только я",0.600000,0.000000,0.200000,0.200000,0.000000,0.0
только не я,0.333333,0.000000,0.000000,0.333333,0.333333,0.0
он,0.000000,0.000000,0.000000,0.000000,0.000000,1.0


In [30]:
frame = {}
for col in table.columns:
    N = len(table)
    df = len(table[col][table[col]!=0])
    series = tfidf[col].apply(lambda x: x*np.log10(N/df))
    frame[col] = series
result = pd.DataFrame(frame)
result

,я,ты,и,только,не,он
я и ты,0.032303,0.132647,0.07395,0.000000,0.00000,0.00000
ты и я,0.032303,0.132647,0.07395,0.000000,0.00000,0.00000
"я, я и только я",0.058146,0.000000,0.04437,0.079588,0.00000,0.00000
только не я,0.032303,0.000000,0.00000,0.132647,0.23299,0.00000
он,0.000000,0.000000,0.00000,0.000000,0.00000,0.69897


## Задание 3 (2 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к классификаторам:   
а) один должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров  
в) у классификатора должно быть задано вручную как минимум 2 параметра  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

In [3]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [23]:
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 1), lowercase=True, stop_words = None, encoding="utf-8")
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 

In [5]:
y = train.toxic.values
y_test = test.toxic.values

In [25]:
clf = LogisticRegression(C=0.1, class_weight='balanced', max_iter = 300)

In [26]:
clf.fit(X, y)
preds_log = clf.predict(X_test)

In [27]:
print(classification_report(y_test, preds_log))

              precision    recall  f1-score   support

         0.0       0.89      0.83      0.86       961
         1.0       0.70      0.80      0.75       481

    accuracy                           0.82      1442
   macro avg       0.80      0.82      0.80      1442
weighted avg       0.83      0.82      0.82      1442



In [40]:
#второй токенизатор для второго классификатора
vectorizer = TfidfVectorizer(analyzer='word', lowercase=True, min_df=2, max_df=0.3, max_features = 10000)
X2 = vectorizer.fit_transform(train.comment)
X2_test = vectorizer.transform(test.comment)

In [41]:
clf_NB = MultinomialNB(alpha = 0.5, fit_prior = True)
clf_NB.fit(X2, y)
preds_NB = clf_NB.predict(X2_test)

print(classification_report(y_test, preds_NB))

              precision    recall  f1-score   support

         0.0       0.84      0.97      0.90       939
         1.0       0.93      0.65      0.77       503

    accuracy                           0.86      1442
   macro avg       0.89      0.81      0.83      1442
weighted avg       0.87      0.86      0.85      1442



## *Задание 4 (2 балла)

Для классификаторов LogisticRegression, Decision Trees, Naive Bayes, Random Forest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.

In [ ]:
# ваш код